<a href="https://colab.research.google.com/github/Aulia-Rnm/BigData/blob/main/pyspark_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2. Pengaturan PySpark di Colab

  Spark ditulis menggunakan bahasa pemrogrraman Scala dan membutuhkan Java Virtual Machine (JVM) untuk menjalankannya. Sebagai langkah pertama, lakukan instalasi Java dengan menuliskan perintah di bawah ini.

- Untuk instalasi Apache Spark, unduh berkas menggunakan perintah wget kemudian
ekstrak dengan perintah tar. Silahkan salin perintah berikut untuk melakukan instalasi.

In [22]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

- Sebagai Langkah lanjutan, dibutuhkan pengaturan terkait Java dan Spark Home. Untuk melakukannya dapat memanfaatkan script pyton. Silahkan masukkan kode berikut ke dalam notebook.

In [23]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

# 3. Konfigurasi PySpark

  Konfigurasi PySpark dapat dilakukan dengan menginstall library findspark yang digunakan untuk mencari lokasi Spark yang terinstall pada sistem. Proses instalasi dapat memanfaatkan perintah pip, silahkan perhatikan perintah di bawah ini.

- Setelah proses instalasi berhasil, lakukan import library dan inisialisasi findspark. Silahkan salin kode berikut ke dalam notebook.

- Koneksi ke dalam spark dapat dilakukan memanfaatkan SparkSession. Salin kode berikut, dimana spark menggunakan port 4050.

In [24]:
!pip install -q findspark

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder\
 .master("local")\
 .appName("Colab")\
 .config('spark.ui.port', '4050')\
 .getOrCreate()

# 4. Memasukkan data ke dalam PySpark
  
  Spark mempunyai beberapa modul untuk membaca data dengan format yang berbeda. Spark secara otomatis akan menentukan tiap tipe data untuk setiap kolom. Data yang akan digunakan sebagai dataset dapat diunduh menggunakan perintah wget. Silahkan perhatikan perintah berikut.

In [25]:
!wget --continue https://raw.githubusercontent.com/dhanifudin/pyspark-demo/main/sample_books.json -O /tmp/sample_books.json

--2022-06-06 13:28:17--  https://raw.githubusercontent.com/dhanifudin/pyspark-demo/main/sample_books.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1565 (1.5K) [text/plain]
Saving to: ‘/tmp/sample_books.json’

/tmp/sample_books.j 100%[===================>]   1.53K  --.-KB/s    in 0s      

2022-06-06 13:28:17 (21.0 MB/s) - ‘/tmp/sample_books.json’ saved [1565/1565]



Data yang telah diunduh dapat dibaca dengan menggunakan kode berikut. Silahkan salin kode berikut ke dalam notebook.

In [26]:
df = spark.read.json("/tmp/sample_books.json")

# 5. Analisa data menggunakan PySpark

  Sebelum dapat menganalisa dataset, perlu mengetahui schema data yang akan diolah. Schema dapat diketahui dengan menggunakan kode berikut. Kode ini memanfaatkan dataframe.

In [27]:
df.printSchema()

root
 |-- author: string (nullable = true)
 |-- edition: string (nullable = true)
 |-- price: double (nullable = true)
 |-- title: string (nullable = true)
 |-- year_written: long (nullable = true)



# 6. Menampilkan data

  Dataset dapat ditampilkan dengan perintah show dari dataframe. Perintah show memiliki parameter berupa jumlah record yang ditampilkan serta opsi truncate. Silahkan salin kode beriikut ke dalam notebook.

In [28]:
df.show(4,False)

+---------------+--------------+-----+----------------+------------+
|author         |edition       |price|title           |year_written|
+---------------+--------------+-----+----------------+------------+
|Austen, Jane   |Penguin       |18.2 |Northanger Abbey|1814        |
|Tolstoy, Leo   |Penguin       |12.7 |War and Peace   |1865        |
|Tolstoy, Leo   |Penguin       |13.5 |Anna Karenina   |1875        |
|Woolf, Virginia|Harcourt Brace|25.0 |Mrs. Dalloway   |1925        |
+---------------+--------------+-----+----------------+------------+
only showing top 4 rows



# 7. Menghitung jumlah dataset
  
  Dataset dapat dihitung menggunakan fungsi count, silahkan salin kode berikut ke dalam untuk mengetahui jumlah dataset yang dimiliki.

In [29]:
df.count()

13

# 8. Menampilkan kolom yang diinginkan

  Dataset dapat dipilih untuk menampilkan data dengan kolom tertentu dengan fungsi select. Untuk menampilkan hanya kolom title, price dan year_written, silahkan salin kode berikut ke dalam colab.

In [30]:
df.select("title", "price", "year_written").show(5)

+----------------+-----+------------+
|           title|price|year_written|
+----------------+-----+------------+
|Northanger Abbey| 18.2|        1814|
|   War and Peace| 12.7|        1865|
|   Anna Karenina| 13.5|        1875|
|   Mrs. Dalloway| 25.0|        1925|
|       The Hours|12.35|        1999|
+----------------+-----+------------+
only showing top 5 rows



# 9. Me-filter dataset
  PySpark dapat juga melakukan filter suatu dataset berdasarkan kondisi yang dibutuhkan. Sebagai contoh: dataset yang ingin ditampilkan adalah buku yang ditulis setelah tahun 1950 dan harganya lebih dari $10. Filter dapat dilakukan dengan menuliskan kode berikut.

In [31]:
df_filtered = df.filter("year_written > 1950 AND price > 10 AND title IS NOT NULL")
df_filtered.select("title", "price", "year_written").show(50, False)

+-----------------------------+-----+------------+
|title                        |price|year_written|
+-----------------------------+-----+------------+
|The Hours                    |12.35|1999        |
|Harry Potter                 |19.95|2000        |
|One Hundred Years of Solitude|14.0 |1967        |
+-----------------------------+-----+------------+



# 10. Penggunaan PySpark SQL functions
  
  PySpark mempunyai fungsi SQL lainnya, misalnya max, aggregate function (groupBy, sum, count dll). Contoh: menampilkan data buku paling mahal, dapat menggunakan fungsi max.

In [32]:
from pyspark.sql import functions as f
maxValue = df_filtered.agg(f.max("price")).collect()[0][0]
print("maxValue: ",maxValue)
df_filtered.select("title","price").filter(df.price == maxValue).show(20, False)

maxValue:  19.95
+------------+-----+
|title       |price|
+------------+-----+
|Harry Potter|19.95|
+------------+-----+



# Tugas

## 1. Tampilkan data buku dengan harga paling murah!

In [33]:
df.filter("price="+str(df.agg(f.min("price")).collect()[0][0])).show()

+----------------+------------+-----+-----------+------------+
|          author|     edition|price|      title|year_written|
+----------------+------------+-----+-----------+------------+
|Dickens, Charles|Random House| 5.75|Bleak House|        1870|
+----------------+------------+-----+-----------+------------+



## 2. Tampilkan jumlah terbit buku dikategorikan setiap tahun ditulis!

In [34]:
df.groupBy('year_written').count().sort(df.year_written).show()

+------------+-----+
|year_written|count|
+------------+-----+
|        1603|    1|
|        1814|    1|
|        1862|    1|
|        1865|    2|
|        1870|    1|
|        1875|    1|
|        1922|    1|
|        1925|    1|
|        1937|    1|
|        1967|    1|
|        1999|    1|
|        2000|    1|
+------------+-----+



## 3. Tampilkan data buku termahal tiap tahun penulisannya!

In [35]:
df.groupBy('year_written').max("price").sort(df.year_written).show()

+------------+----------+
|year_written|max(price)|
+------------+----------+
|        1603|      7.95|
|        1814|      18.2|
|        1862|      7.75|
|        1865|      12.7|
|        1870|      5.75|
|        1875|      13.5|
|        1922|      29.0|
|        1925|      25.0|
|        1937|     27.45|
|        1967|      14.0|
|        1999|     12.35|
|        2000|     19.95|
+------------+----------+



## 4. Tampilkan data buku termurah tiap tahun penulisannya!

In [36]:
df.groupBy('year_written').min("price").sort(df.year_written).show()

+------------+----------+
|year_written|min(price)|
+------------+----------+
|        1603|      7.95|
|        1814|      18.2|
|        1862|      7.75|
|        1865|      5.76|
|        1870|      5.75|
|        1875|      13.5|
|        1922|      29.0|
|        1925|      25.0|
|        1937|     27.45|
|        1967|      14.0|
|        1999|     12.35|
|        2000|     19.95|
+------------+----------+

